In [1]:
# import pandas as pd
# import os
# import glob
# import concurrent.futures
# import time

# directory = 'C:\\Users\\minhtriet.pham\\Desktop\\Data Invoice\\NAM 2023\\Month 3'
# directory2 = 'C:\\Users\\minhtriet.pham\\Desktop\\Data Invoice\\NAM 2023\\Month 3'
# cate_dir = 'C:\\Users\\minhtriet.pham\\Desktop\\DataCate.xlsx'
# cate_dir2 = 'C:\\Users\\minhtriet.pham\\Desktop\\DataCate.xlsx'

# # list of Excel files to read data from
# if os.path.exists(os.path.join(directory, '230301 Payment Summary - 2023-03-02T084245.428.xlsx')):
#     excel_files = glob.glob(os.path.join(directory, '*.xlsx'))
# else:
#     excel_files = glob.glob(os.path.join(directory2, '*.xlsx'))
# try:
#     # # use cols 0, Barcode; 2, Category Name
#     # df2 = pd.read_excel(cate_dir, sheet_name='Sheet1', usecols=[0,2])
#     # use cols 0, Barcode; 3, SubCategory Name
#     df2 = pd.read_excel(cate_dir, sheet_name='Sheet1', usecols=[0,3])
#     pass
# except FileNotFoundError:
#     # df2 = pd.read_excel(cate_dir2, sheet_name='Sheet1', usecols=[0,2])
#     # use cols 0, Barcode; 3, SubCategory Name
#     df2 = pd.read_excel(cate_dir2, sheet_name='Sheet1', usecols=[0,3])
#     pass
# # create an empty list to store the dataframes
# # dfs = [] sho

# outputfile =  'dataM1.csv'
# # def excel2csv
# def excel2csv(inputfile, outputfile):
    
#     # Check if exist file_path
#     if not os.path.exists(outputfile):
#         with open(outputfile, 'w') as f:
#             pass

#     # Read the first line of the file
#     with open(outputfile, 'r') as f:
#         first_line = f.readline().strip()
#     # Check if the first line contains the expected column names
#     has_headers = 'InvoiceID' in first_line
#     df = pd.read_excel(inputfile, sheet_name='Sheet2', skiprows=2, usecols=[4,7,8,14])
#     # df2 Category
#     # # merge with Category, on "Barcode"
#     # df = df.merge(df2, on='Barcode', how='left')
#     # merge with Sub Category, on "Barcode"
#     df = df.merge(df2, on='Barcode', how='left')
    
#     # Check if the first line contains the expected column names
#     df.to_csv(outputfile, mode='a', header= not has_headers, index=False)


# # loop through the list of Excel files
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     [executor.submit(excel2csv, file, outputfile) for file in excel_files]
#     # for file in excel_files:
#     #     executor.submit(excel2csv,file)

In [5]:
import pandas as pd

# Sample data
dtypes = {'InvoiceID': str, 'Barcode': str, 'Sum of Qty': str, 'Total Exclude VAT': str, 'SubCategory Name': str}
# df = pd.read_csv(r'C:\Users\minhtriet.pham\Desktop\New folder\Quantity\dataM1.csv', dtype=dtypes, on_bad_lines='warn')
try:
    df = pd.read_csv(r'C:\Users\trieu.pham\OneDrive - BTM Global Consulting\Projects\data csv\dataM1.csv', dtype=dtypes, on_bad_lines='warn')
except FileNotFoundError:
    df = pd.read_csv(r'/Users/trieupham/Library/CloudStorage/OneDrive-BTMGlobalConsulting/Projects/data csv/dataM1.csv', dtype=dtypes, on_bad_lines='warn')
except:
    df = pd.read_csv(r'C:\Users\minhtriet.pham\Desktop\New folder\Quantity\dataM1.csv', dtype=dtypes, on_bad_lines='warn')

df['Sum of Qty'] = pd.to_numeric(df['Sum of Qty'], errors='coerce').fillna(0).astype(int)
df['Total Exclude VAT'] = pd.to_numeric(df['Total Exclude VAT'], errors='coerce').fillna(0).astype(float)
df['SubCategory Name'] = df['SubCategory Name'].astype(str)
# df['Category Name'] = df['Category Name'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5377868 entries, 0 to 5377867
Data columns (total 5 columns):
 #   Column             Dtype  
---  ------             -----  
 0   InvoiceID          object 
 1   Barcode            object 
 2   Sum of Qty         int64  
 3   Total Exclude VAT  float64
 4   SubCategory Name   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 205.1+ MB


In [ ]:
# Filter InvoiceIDs where sum of Qty is 0 for all barcodes
invoice_ids_with_zero_qty = df.groupby('InvoiceID').filter(lambda x: x['Sum of Qty'].sum() == 0)['InvoiceID'].unique()

In [ ]:
# Filter the original data frame based on the InvoiceIDs
df_filtered_invoice_id_zero_qty = df[df['InvoiceID'].isin(invoice_ids_with_zero_qty)]

df_filtered_invoice_id_zero_qty.sort_values(by=['InvoiceID', 'Barcode'])


In [ ]:
df

In [ ]:
# Find records in df that are not included in df_filtered_invoice_id_zero_qty
df_filtered = pd.merge(df, df_filtered_invoice_id_zero_qty, on=['InvoiceID', 'Barcode', 'Sum of Qty', 'Total Exclude VAT', 'SubCategory Name'], how='left', indicator=True)
df_filtered = df_filtered[df_filtered['_merge'] == 'left_only'].drop('_merge', axis=1)
df_filtered

In [ ]:
import pandas as pd
import numpy as np

# filter out rows where 'Sum of Qty' or 'Total Exclude VAT' are zero
# df_filtered = df[(df['Sum of Qty'] > 0) & (df['Total Exclude VAT'] > 0)]

group_by_invoice = df_filtered.dropna().groupby('InvoiceID').agg({'Barcode': lambda x: ', '.join(x.astype(str)), 
                                      'Sum of Qty': 'sum', 
                                      'Total Exclude VAT': 'sum', 
                                      'SubCategory Name': lambda x: ', '.join(x.astype(str))}).reset_index()
group_by_invoice

In [ ]:
# filter group_by_invoice by upper_bound and lower_bound
upper_bound = 50000000
lower_bound = 1000

group_by_invoice_filtered = group_by_invoice[(group_by_invoice['Total Exclude VAT'] >= lower_bound) & (group_by_invoice['Total Exclude VAT'] <= upper_bound)]

group_by_invoice_filtered

In [ ]:
import statistics
from scipy import stats
# Calculate the statistical measures using describe() method
stat = group_by_invoice_filtered.describe()

# Calculate the median and mode of the 'Total Exclude VAT' column
mean = statistics.mean(group_by_invoice_filtered['Total Exclude VAT'])
median = statistics.median(group_by_invoice_filtered['Total Exclude VAT'])
mode = group_by_invoice_filtered['Total Exclude VAT'].mode()[0]
std = statistics.stdev(group_by_invoice_filtered['Total Exclude VAT'])
Sum = group_by_invoice_filtered['Total Exclude VAT'].sum()
min_val = group_by_invoice_filtered['Total Exclude VAT'].min()
max_val = group_by_invoice_filtered['Total Exclude VAT'].max()
q1 = np.percentile(group_by_invoice_filtered['Total Exclude VAT'], 25)
q3 = np.percentile(group_by_invoice_filtered['Total Exclude VAT'], 75)
# Extract the desired values from the summary
mean1 = stat.loc['mean']
std1 = stat.loc['std']
min_val1 = stat.loc['min']
max_val1 = stat.loc['max']

# Print the results
print("Mean:", mean)
print("Median:", median)
print("Mode:", mode)
print("Standard Deviation:", std)
print ("Sum:", Sum)
print("Minimum Value:", min_val)
print("Maximum Value:", max_val)
print('Q1:', q1)
print('Q3:', q3)

# print("Mode of Total Exculde VAT", group_by_invoice['Total Exclude VAT'].mode()[0])

# Create a DataFrame with the calculated statistical measures 
summary_statistic_df = pd.DataFrame({'Mean': [mean],
                                     'Median': [median],
                                     'Mode': [mode],
                                     'Standard Deviation': [std],
                                     'Sum': [Sum],
                                     'Minimum Value': [min_val],
                                     'Maximum Value': [max_val]})

# Export the DataFrame to an Excel file
# summary_statistic_df.to_excel('Summary Statistic M3.xlsx', index=False)

In [ ]:
import numpy as np
import pandas as pd

# Update bins and labels for the new range concept
bins = np.concatenate(([0], np.arange(0, 16000, 16000), np.arange(16000, 30000, 14000), np.arange(30000, 53000, 23000), np.arange(53000, 350001, 297000), [np.inf]))
bins = np.unique(bins)  # Remove duplicate edges

# Update labels
labels = ['{} to {}' .format(i, i+15999) for i in range(0, 16000, 16000)] + ['{} to {}' .format(i, i+13999) for i in range(16000, 30000, 14000)] + ['{} to {}' .format(i, i+22999) for i in range(30000, 53000, 23000)] + ['{} to {}' .format(i, i+296999) for i in range(53000, 350000, 297000)]+ ['>=350000']

def grouped_data(df_filtered):
    df_filtered['Basket Value'] = pd.cut(df_filtered['Total Exclude VAT'], bins=bins, labels=labels)
    grouped_data = df_filtered.groupby('Basket Value')['Total Exclude VAT'].agg(['count','mean','median'])

    # Add a new column for Count x Mean
    grouped_data['Value'] = grouped_data['count'] * grouped_data['mean']

    # Update x-axis labels
    grouped_data.index = grouped_data.index.astype(str).str.replace(',', '')

    return grouped_data

grouped_data = grouped_data(group_by_invoice_filtered)

grouped_data


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# Calculate total value
total_value = grouped_data['Value'].sum()

# Add a new column for %Value in total
grouped_data['%Value in total'] = grouped_data['Value'] / total_value * 100

# Plot histogram for count
fig, ax1 = plt.subplots()
n, bins, patches = ax1.hist(df_filtered['Total Exclude VAT'], bins=bins, cumulative=False, alpha=0.7, color='blue', edgecolor='black')
ax1.set_xlabel('Basket Value')
ax1.set_ylabel('Count (in units)')
ax1.set_title('Basket Value Distribution - Bill')
ax1.legend(['Count'], loc='upper left')
ax1.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(int(x))))

# Plot histogram for value
fig, ax2 = plt.subplots()
n2, bins2, patches2 = ax2.hist(df_filtered['Total Exclude VAT'], bins=bins, cumulative=False, alpha=0.7, color='red', edgecolor='black', weights=df_filtered['Total Exclude VAT'])
ax2.set_xlabel('Basket Value')
ax2.set_ylabel('Value (in VND)')
ax2.set_title('Basket Value Distribution - Value')
ax2.legend(['Value'], loc='upper left')
ax2.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(int(x))))

# Add percentage value labels to the top of each column
for i in range(len(patches2)):
    height = patches2[i].get_height()
    percentage = height / total_value * 100
    ax2.annotate(f'{percentage:.1f}%', xy=(patches2[i].get_x() + patches2[i].get_width() / 2, height), xytext=(0, 3), textcoords='offset points', ha='center')

# Highlight columns with cumulative %Value in total up to 80%
cumulative_percentage = 0
for i, patch in enumerate(patches2):
    cumulative_percentage += grouped_data['%Value in total'].iloc[i]
    if cumulative_percentage <= 80:
        patch.set_fc('green')
    else:
        patch.set_fc('red')

plt.show()

# Display grouped data with additional column
grouped_data['Count'] = grouped_data['count'].apply(lambda x: '{:,.0f}'.format(x))
grouped_data['Mean'] = grouped_data['mean'].apply(lambda x: '{:,.0f}'.format(x))
grouped_data['Median'] = grouped_data['median'].apply(lambda x: '{:,.0f}'.format(x))
grouped_data['Count x Mean'] = grouped_data['Value'].apply(lambda x: '{:,.0f}'.format(x))
grouped_data = grouped_data[['Count', 'Mean', 'Median', 'Value', '%Value in total']]
print(grouped_data)

In [ ]:
# grouped_data.to_csv('groupdataM1.csv')

In [ ]:
# Apply string slicing to create new columns 'StorID' and 'BusinessDate'
group_by_invoice_filtered['StoreID'] = group_by_invoice_filtered['InvoiceID'].map(lambda x: x[:6])
group_by_invoice_filtered['BusinessDate'] = group_by_invoice_filtered['InvoiceID'].map(lambda x: x[8:14])

# Reorder the columns
group_by_invoice_filtered = group_by_invoice_filtered.reindex (columns=['InvoiceID','StoreID','BusinessDate','Barcode','Sum of Qty','Total Exclude VAT','SubCategory Name'])

# Show the first 5 rows
group_by_invoice_filtered.head(5)

In [ ]:
# Group by StoreID and aggregate by 'InvoiceID', 'Total Exclude VAT', 'Sum of Qty' and 'BusinessDate'

grouped_by_store_net = group_by_invoice_filtered.groupby('StoreID').agg({'InvoiceID': 'nunique','Total Exclude VAT': 'sum', 'Sum of Qty': 'sum', 'BusinessDate': 'nunique'})

# Calculate the ADS, ADQ and ASI

grouped_by_store_net['ADS'] = grouped_by_store_net['Total Exclude VAT'] / grouped_by_store_net['BusinessDate']
grouped_by_store_net['ADQ'] = grouped_by_store_net['Sum of Qty'] / grouped_by_store_net['BusinessDate']
grouped_by_store_net['ASI'] = grouped_by_store_net['Total Exclude VAT'] / grouped_by_store_net['Sum of Qty']
grouped_by_store_net['CUS'] = grouped_by_store_net['InvoiceID'] / grouped_by_store_net['BusinessDate']
grouped_by_store_net['BV'] = grouped_by_store_net['ADS'] / grouped_by_store_net['CUS']
grouped_by_store_net['Avg_Items_Per_Bill'] = grouped_by_store_net['ADQ'] / grouped_by_store_net['CUS']

# Reset the index to make 'StoreID' a regular column

grouped_by_store_net = grouped_by_store_net.reset_index()

grouped_by_store_net



In [ ]:
# Create a new column 'Basket Value' based on the column 'Total Exclude VAT'
group_by_invoice_filtered['Basket Value'] = pd.cut(group_by_invoice_filtered['Total Exclude VAT'], bins=bins, labels=labels)

# Create a new dataframe 'group_basket_store' = group_by_invoice_filtered
group_basket_store = group_by_invoice_filtered

# Apply string slicing to create new column 'StoreID'
group_basket_store['StoreID'] = group_basket_store['InvoiceID'].map(lambda x: x[:6])

# Keep only the columns 'InvoiceID', 'Sum of Qty', 'Total Exclude VAT', 'Basket Value' and 'StoreID'
group_basket_store = group_basket_store[['InvoiceID','Sum of Qty','Total Exclude VAT','Basket Value','StoreID']].reindex(columns=['Basket Value','StoreID','InvoiceID','Sum of Qty','Total Exclude VAT'])

# Group by 'Basket Value' and 'StoreID' and aggregate by 'Total Exclude VAT'
group_basket_store = group_basket_store.groupby(['Basket Value','StoreID']).agg({'Total Exclude VAT': ['count', 'mean', 'median']})

# Access columns using tuple indexing and compute 'Value' column
group_basket_store.loc[:, ('Total Exclude VAT','Value')] = group_basket_store[('Total Exclude VAT','count')] * group_basket_store[('Total Exclude VAT','mean')]

group_basket_store.reset_index(inplace=True)

group_basket_store

In [ ]:
df_filtered

In [ ]:
try:
    df_read_cate = pd.read_excel(r'C:\Users\trieu.pham\OneDrive - BTM Global Consulting\Projects\github\python\Python\Data Analysis\DataCate.xlsx', sheet_name='Sheet1', usecols=[0,2])
except:
    df_read_cate = pd.read_excel(r'C:\Users\Trieu Pham\OneDrive - BTM Global Consulting\Projects\github\python\Python\Data Analysis\DataCate.xlsx', sheet_name='Sheet1', usecols=[0,2])
df_read_cate['Barcode'] = df_read_cate['Barcode'].astype(str)
df_filtered = pd.merge(df_filtered, df_read_cate, on='Barcode', how='left')

df_filtered

In [ ]:
group_basket_cate_subcate = df_filtered

# filter group_basket_cate_subcate by upper_bound and lower_bound
upper_bound = 50000000
lower_bound = 1000
group_basket_cate_subcate = group_basket_cate_subcate[(group_basket_cate_subcate['Total Exclude VAT'] < upper_bound) & (group_basket_cate_subcate['Total Exclude VAT'] > lower_bound)]
group_basket_cate_subcate['Basket Value'] = pd.cut(df_filtered['Total Exclude VAT'], bins=bins, labels=labels)

# group_basket_cate_subcate.to_csv('group_basket_cate.csv')
group_basket_cate_subcate

In [ ]:

queries = group_basket_cate_subcate[(group_basket_cate_subcate['Total Exclude VAT'] < 1000)]

group_basket_cate_subcate['StoreID'] = group_basket_cate_subcate['InvoiceID'].map(lambda x: x[:6])

group_basket_cate_subcate

In [ ]:
# Get the sum of 'Total Exclude VAT', grouped by 'Basket Value', 'Category Name' and 'SubCategory Name'
total_exclude_vat_by_category = group_basket_cate_subcate.groupby(['Basket Value','Category Name','SubCategory Name']).agg({'Total Exclude VAT':'sum'}).reset_index()

# total_exclude_vat_by_category.sort_values(by=['Total Exclude VAT'], ascending=True)

# Get the top 5 categories based on the sum of 'Total Exclude VAT', grouped by 'Basket Value'
top_5_categories = total_exclude_vat_by_category.groupby('Basket Value').apply(lambda x: x.nlargest(5, 'Total Exclude VAT')).reset_index(drop=True)

# Get the top 10 subcategories based on the sum of 'Total Exclude VAT', grouped by 'Basket Value'
top_10_subcategories = total_exclude_vat_by_category.groupby('Basket Value').apply(lambda x: x.nlargest(10, 'Total Exclude VAT')).reset_index(drop=True)

In [ ]:
top_5_categories

In [ ]:
top_5_categories

In [ ]:
top_10_subcategories

In [ ]:
# Get the sum of 'Total Exclude VAT', grouped by 'Basket Value', 'Category Name' and 'StoreID'
total_exclude_vat_by_category_store = group_basket_cate_subcate.groupby(['Basket Value','Category Name','StoreID']).agg({'Total Exclude VAT':'sum'}).reset_index()

# Get the top 5 categories based on the sum of 'Total Exclude VAT', grouped by 'Basket Value' for each store
top_5_categories_store = total_exclude_vat_by_category_store.groupby(['Basket Value','StoreID']).apply(lambda x: x.nlargest(5, 'Total Exclude VAT')).reset_index(drop=True)

In [ ]:
top_5_categories_store
# top_5_categories_store.to_csv('top_5_categories_store.csv')

In [ ]:
# Get the sum of 'Total Exclude VAT', grouped by 'Basket Value', 'SubCategory Name' and 'StoreID'
total_exclude_vat_by_subcategory_store = group_basket_cate_subcate.groupby(['Basket Value','SubCategory Name','StoreID']).agg({'Total Exclude VAT':'sum'}).reset_index()

# Get the top 10 subcategories based on the sum of 'Total Exclude VAT', grouped by 'Basket Value' for each store
top_10_subcategories_store = total_exclude_vat_by_subcategory_store.groupby(['Basket Value','StoreID']).apply(lambda x: x.nlargest(10, 'Total Exclude VAT')).reset_index(drop=True)

In [ ]:
top_10_subcategories_store
# top_10_subcategories_store.to_csv('top_10_subcategories_store.csv')

In [ ]:
import numpy as np
from sklearn.linear_model import RANSACRegressor

# StoreID and BusinessDate added to the filter dataframe
df_filtered['InvoiceID'] = df_filtered['InvoiceID'].astype(str)
df_filtered['StoreID'] = df_filtered['InvoiceID'].apply(lambda x: x[:6])
df_filtered['BusinessDate'] = df_filtered['InvoiceID'].apply(lambda x: x[8:14])

# Group by Basket Value and StoreID and calculate count and mean
grouped_data_store = df_filtered.groupby(['InvoiceID', 'StoreID']).agg({'InvoiceID': 'count', 'Total Exclude VAT': 'mean'})
grouped_data_store.rename(columns={'InvoiceID': 'count', 'Total Exclude VAT': 'mean'}, inplace=True)
grouped_data_store.reset_index(inplace=True)

# Calculate the 'value' column as the product of 'mean' and 'count'
grouped_data_store['Value'] = grouped_data_store['mean'] * grouped_data_store['count']

# Drop rows with NaN or 0 values in 'mean' or 'Value' columns
grouped_data_store.dropna(subset=['mean', 'Value'], inplace=True)
grouped_data_store = grouped_data_store[(grouped_data_store['mean'] != 0) & (grouped_data_store['Value'] != 0)]

# Fit RANSAC Regression model to 'Value' and 'mean' columns
X = grouped_data_store[['Value']].values
y = grouped_data_store['mean'].values

regressor = RANSACRegressor()
regressor.fit(X, y)

# Predict the mean values using the fitted model
y_pred = regressor.predict(X)

# Calculate the residuals
residuals = y - y_pred

# Set a threshold for identifying outliers
threshold = 1.5 * np.median(np.abs(residuals))

# Identify outliers based on residuals
outliers = grouped_data_store[residuals > threshold]

# Remove outliers from the grouped_data_store DataFrame
grouped_data_store_clean = grouped_data_store[residuals <= threshold]

# Show the cleaned grouped data columns
print(grouped_data_store_clean)


In [ ]:
import numpy as np

# Sort the 'grouped_data_store' DataFrame by the 'value' column in descending order
grouped_data_store.sort_values('Value', ascending=False, inplace=True)

# Calculate the cumulative sum of the 'value' column
grouped_data_store['cumulative_sum'] = grouped_data_store['Value'].cumsum()

# Calculate the cumulative distribution percentage
grouped_data_store['cumulative_percentage'] = grouped_data_store['cumulative_sum'] / grouped_data_store['Value'].sum()

# Filter rows where the cumulative percentage is less than or equal to 0.8
filtered_data = grouped_data_store[grouped_data_store['cumulative_percentage'] >= 0.8]

# Print the filtered results
print(filtered_data)